In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sim.Box2DSim import Box2DSim, InlineTestPlotter
from DMp.dmp import DMP


In [18]:
# define a single step of the simulation
class Simulation:
    def __init__(self):
        self.dmps = [ DMP(n = 50, s = 0, g = 1, stime = 102)
                     for x in range(5)]
        for d in self.dmps:
            d.rollout()
        self.t = 0

        
    def __call__(self, sim):
        # set the move of  the "Arm1_to_Arm2" joint to 
        # the -np.pi/2.0 target position

        k = self.t
        
        sim.move("Arm1_to_Arm2", -0.5*np.pi*self.dmps[0].S["y"][k])
        sim.move("Arm2_to_Arm3", -0.5*np.pi*self.dmps[1].S["y"][k])
        sim.move("Arm2_to_Arm3", -0.5*np.pi*self.dmps[2].S["y"][k])
        sim.move("Arm3_to_Claw10", -0.5*np.pi*(1-self.dmps[3].S["y"][k]))
        sim.move("Arm3_to_Claw20", 0.5*np.pi*(1-self.dmps[3].S["y"][k]))
                 
        # do the movement
        sim.step()
        contacts = [
            sim.contacts("claw10", "Object"),
            sim.contacts("claw11", "Object"),
            sim.contacts("claw20", "Object"),
            sim.contacts("claw21", "Object")]
            
        
        self.t += 1
        

In [19]:
# create the simulator
inline_sim = Box2DSim("body2d.json")
simulation = Simulation()
# create the plot environment
plotter = InlineTestPlotter(inline_sim, sim_step=simulation)
# run the simulation so to produce the video
ani = plotter.makeVideo(frames=100)

In [20]:
# show the video of the simulation
from IPython.display import HTML
HTML(ani.to_html5_video())